In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion")

dataset = pd.read_csv("emotions/train.csv")

dataset.head()

,text,emotion,category
0,Жахливе місце. Жахлива черга. Ніхто нічого під...,Anger,Complaint / Dissatisfaction
1,"Жахливе відношення до людей,які записані на о...",Anger,Complaint / Dissatisfaction
2,"Як були черги величезні та багатогодинні, так ...",Anger,Complaint / Dissatisfaction
3,Сьогодні відвідала ЦНАП(була екскурсія від іні...,Happiness,Gratitude / Positive Feedback
4,Пиляток на всьому. Ряд об'єктів почали і не до...,Anger,Complaint / Dissatisfaction


In [24]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

In [27]:
# ============================================================

MODEL = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

tokenizer("Hello, how are you?")

token_count = [len(tokenizer.encode(text, add_special_tokens=True)) for text in dataset["text"]]

#token distribution summary
pd.Series(token_count).describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    8106.000000
mean       41.979151
std        23.503552
min        11.000000
25%        24.000000
50%        37.000000
75%        58.000000
90%        70.000000
95%        77.000000
99%        94.000000
max       437.000000
dtype: float64

In [34]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nazara/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

['Так само, як було багато і багато годин в черзі, і більше не було людей в черзі до мого питання 20 жовтня 7 листопада, і результатом було те, що я лише 242 відповідно до свого питання...']


In [ ]:
#create bag of words model and logreg pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
#sklearn.svm.SVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
 
pipeline = Pipeline([
    # ("vectorizer", CountVectorizer(ngram_range=(1, 1))),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', LogisticRegression(max_iter=1000,class_weight='balanced'))
    # ("classifier", SVC(class_weight='balanced'))
])
pipeline.fit(train_df['text_processed'], train_df['emotion'])
val_preds = pipeline.predict(val_df['text_processed'])

from sklearn.metrics import classification_report
print(classification_report(val_df['emotion'], val_preds))

              precision    recall  f1-score   support

       Anger       0.69      0.83      0.75       299
     Disgust       1.00      0.17      0.29        18
        Fear       1.00      0.12      0.22         8
   Happiness       0.94      0.88      0.91      1082
     Neutral       0.45      0.55      0.50       148
     Sadness       0.22      0.27      0.24        56
    Surprise       0.00      0.00      0.00        10

    accuracy                           0.80      1621
   macro avg       0.61      0.40      0.42      1621
weighted avg       0.82      0.80      0.80      1621



In [2]:
dataset.emotion.value_counts()

emotion
Happiness    5290
Anger        1585
Neutral       782
Sadness       297
Disgust        74
Surprise       40
Fear           38
Name: count, dtype: int64

In [3]:
import stanza

# stanza.download('uk')
nlp = stanza.Pipeline(lang='uk', processors='tokenize, lemma')


doc = nlp("Діти гралися в парку, а мама йшла додому.")


def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return ' '.join(lemmas)

/home/nazara/UCU/NLP/ASSIGNMENT1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-02 10:56:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-12-02 10:56:12 INFO: Downloaded file to /home/nazara/stanza_resources/resources.json
2025-12-02 10:56:12 WARNING: Language uk package default expects mwt, which has been added
2025-12-02 10:56:12 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| lemma     | iu_nocharlm |

2025-12-02 10:56:12 INFO: Using device: cuda
2025-12-02 10:56:12 INFO: Loading: tokenize
2025-12-02 10:56:13 INFO:

In [4]:
alt.Chart(dataset).mark_bar().encode(
    x='emotion',
    y='count()',
    color='emotion',
).properties(
    width=600,
    height=400
)

alt.Chart(...)

In [5]:
alt.Chart(dataset).mark_bar().encode(
    x='category',
    y='count()',
    color='category',
).properties(
    width=600,
    height=400,
)

alt.Chart(...)

In [6]:
with open("resources/stopwords_ua.txt", "r", encoding="utf-8") as file:
    stopwords_ua = [line.strip() for line in file.readlines()]

In [7]:
#emojis in text:

#detect emojis as unicode characters
import re
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "]+",
    flags=re.UNICODE,
)

# find and print all emojis in the 'text' column
dataset['emojis'] = dataset['text'].apply(lambda x: emoji_pattern.findall(x))


In [8]:
#value count for emojis
emojis = dataset['emojis'].to_list()
from collections import Counter
emoji_counts = Counter()
for emoji_list in emojis:
    emoji_counts.update(emoji_list)

emoji_counts.most_common(20)

[('👍', 44),
 ('😊', 22),
 ('😉', 15),
 ('🔥', 15),
 ('😍', 11),
 ('🏻', 9),
 ('👌', 9),
 ('😁', 9),
 ('🙂', 8),
 ('👍👍👍', 7),
 ('😡', 7),
 ('🔺', 7),
 ('🙏', 6),
 ('💪', 6),
 ('💯', 5),
 ('😀', 4),
 ('😎', 4),
 ('😄', 4),
 ('🇺🇦', 4),
 ('😂', 4)]

In [18]:

def preprocess_text(text):
    text = text.lower()
    text = lemmatize_text(text)

    date_pattern = r'\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b'

    text = re.sub(date_pattern, 'DATE', text)

    #remove punct 
    for char in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~':
        text = text.replace(char, " ")
    
    tokens = text.split()
    tokens = [token for token in tokens if token not in stopwords_ua]
    return " ".join(tokens)


dataset['text_processed'] = dataset['text'].apply(lambda x : preprocess_text(x))
# dataset['text_no_lemmas'] = dataset['text'].apply(lambda x: preprocess_text(x))
dataset

,text,emotion,category,emojis,text_no_lemmas,text_processed
0,Жахливе місце. Жахлива черга. Ніхто нічого під...,Anger,Complaint / Dissatisfaction,[],жахливе місце жахлива черга підказати,жахливе місце жахливий черга підказати
1,"Жахливе відношення до людей,які записані на о...",Anger,Complaint / Dissatisfaction,[],жахливе відношення людей записані чекають наба...,жахливе відношення записаний чекають набагато ...
2,"Як були черги величезні та багатогодинні, так ...",Anger,Complaint / Dissatisfaction,[],черги величезні багатогодинні залишилися ганьб...,черга величезня багатогодинний залишитися гань...
3,Сьогодні відвідала ЦНАП(була екскурсія від іні...,Happiness,Gratitude / Positive Feedback,[],відвідала цнап екскурсія ініверситету розповід...,відвідал екскурсія ініверситету розповідати ко...
4,Пиляток на всьому. Ряд об'єктів почали і не до...,Anger,Complaint / Dissatisfaction,[],пиляток ряд єктів почали доробили,пиляток ряд об’єкт почати доробити
...,...,...,...,...,...,...
8101,Один із найкращих коледжів у Харкові. Якщо щос...,Happiness,Gratitude / Positive Feedback,[],найкращих коледжів харкові незрозуміло розкажу...,найкращий коледжі харк незрозуміло розказати д...
8102,03.09.2020\nЦей момент коли вистоюєш чергу. Що...,Anger,Complaint / Dissatisfaction,[],DATE момент вистоюєш чергу записатись прийом в...,03 09 2020 момент вистоюєш черга записатись пр...
8103,У Запоріжжі неможливо жити з-за жахливого смор...,Anger,Complaint / Dissatisfaction,[],запоріжжі неможливо жити жахливого смороду про...,запоріжжя неможливо жити жахливо сморід промпі...
8104,"Чудовий навчальний заклад, що дійсно навчає та...",Happiness,Gratitude / Positive Feedback,[],чудовий навчальний заклад навчає дає знання ви...,чудовий навчальний заклад навчати знання викла...


In [22]:
dataset.sample(frac=1)

,text,emotion,category,emojis,text_no_lemmas,text_processed
823,Просто жахливий заклад. Наплювальне ставлення ...,Anger,Complaint / Dissatisfaction,[],жахливий заклад наплювальне ставлення студенті...,жахливий заклад наплювальне ставлення студент ...
3934,Будівля обласної адміністрації в індустріально...,Neutral,Neutral Comment,[],будівля обласної адміністрації індустріальному...,будівля обласна адміністрація індустріальний с...
3366,Все чудово! Оформлювали паспорт. Швидко та про...,Happiness,Gratitude / Positive Feedback,[],чудово оформлювали паспорт швидко професійно,чудово оформлювалий паспорт швидко професійно
5119,"Співробітник центру Кіра, дуже ввічливо і опер...",Happiness,Gratitude / Positive Feedback,[],співробітник центру кіра ввічливо оперативно в...,співробітник центр кіра ввічливо оперативний в...
5477,"Володимире, для того, щоб нотаріуси могли прий...",Neutral,Suggestion / Idea,[],володимире нотаріуси приймати id карту закордо...,володимире нотаріти приймати id карт закордонн...
...,...,...,...,...,...,...
5115,Звертався з проханням консультації стосовно зе...,Happiness,Gratitude / Positive Feedback,[],звертався проханням консультації земельної діл...,звертався прохання консультація земельний діля...
7820,"Швидко оформили документи на виписку, зробили ...",Happiness,Gratitude / Positive Feedback,[],швидко оформили документи виписку зробили обіц...,швидко оформити документ виписок зробити обіця...
6028,"Дякую, Олександру Павловичу, за надання кваліф...",Happiness,Gratitude / Positive Feedback,[],олександру павловичу надання кваліфікованої юр...,дякувати олександати павлович надання кваліфік...
194,Від імені учнів 8-А класу школи №24 м. Харкова...,Happiness,Gratitude / Positive Feedback,[],імені учнів 8 класу школи №DATE харкова хочеть...,ім’я учень 8 клас школа № 24 харкова хотітися ...


In [15]:
train_df = dataset.sample(frac=0.8, random_state=42)
val_df = dataset.drop(train_df.index)

In [16]:
#create bag of words model and logreg pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
#sklearn.svm.SVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    # ("vectorizer", CountVectorizer(ngram_range=(1, 1))),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', LogisticRegression(max_iter=1000,class_weight='balanced'))
    # ("classifier", SVC(class_weight='balanced'))
])
pipeline.fit(train_df['text_processed'], train_df['emotion'])
val_preds = pipeline.predict(val_df['text_processed'])

from sklearn.metrics import classification_report
print(classification_report(val_df['emotion'], val_preds))

              precision    recall  f1-score   support

       Anger       0.69      0.83      0.75       299
     Disgust       1.00      0.17      0.29        18
        Fear       1.00      0.12      0.22         8
   Happiness       0.94      0.88      0.91      1082
     Neutral       0.45      0.55      0.50       148
     Sadness       0.22      0.27      0.24        56
    Surprise       0.00      0.00      0.00        10

    accuracy                           0.80      1621
   macro avg       0.61      0.40      0.42      1621
weighted avg       0.82      0.80      0.80      1621



In [ ]:
# list(pipeline.named_steps['vectorizer'].get_feature_names_out())

In [17]:
from sklearn.utils import resample
import pandas as pd

train_df_balanced_parts = []

TARGET_SIZES = {
    "Happiness": 1500,
    "Anger": 1585,
    "Neutral": 782,
    "Sadness": 300,
    "Disgust": 300,
    "Surprise": 300,
    "Fear": 300,
}

for label, target in TARGET_SIZES.items():
    df_label = train_df[train_df["emotion"] == label]
    
    if len(df_label) >= target:
        # undersample majority class (only Happiness)
        df_resampled = resample(
            df_label,
            replace=False,
            n_samples=target,
            random_state=42
        )
    else:
        # oversample minority classes
        df_resampled = resample(
            df_label,
            replace=True,
            n_samples=target,
            random_state=42
        )

    train_df_balanced_parts.append(df_resampled)

train_df_balanced = pd.concat(train_df_balanced_parts)

print(train_df_balanced['emotion'].value_counts())
pipeline.fit(train_df_balanced['text_processed'], train_df_balanced['emotion'])

val_preds_balanced = pipeline.predict(val_df['text_processed'])
print(classification_report(val_df['emotion'], val_preds_balanced))


emotion
Anger        1585
Happiness    1500
Neutral       782
Sadness       300
Disgust       300
Surprise      300
Fear          300
Name: count, dtype: int64
              precision    recall  f1-score   support

       Anger       0.68      0.83      0.75       299
     Disgust       0.50      0.11      0.18        18
        Fear       0.50      0.12      0.20         8
   Happiness       0.93      0.88      0.90      1082
     Neutral       0.45      0.54      0.49       148
     Sadness       0.20      0.16      0.18        56
    Surprise       0.00      0.00      0.00        10

    accuracy                           0.80      1621
   macro avg       0.46      0.38      0.39      1621
weighted avg       0.80      0.80      0.80      1621



In [ ]:
heatmap_data = (
    dataset
    .groupby(['category', 'emotion'])
    .size()
    .reset_index(name='count')
)

heatmap = (
    alt.Chart(heatmap_data)
    .mark_rect()
    .encode(
        x=alt.X('category:N', sort='-y', title='Category'),
        y=alt.Y('emotion:N', sort='-x', title='Emotion'),
        color=alt.Color('count:Q', scale=alt.Scale(scheme='blues')),
        tooltip=['category', 'emotion', 'count']
    )
    .properties(
        width=800,
        height=800,
        title='Category × Emotion Heatmap'
    )
)

text_labels = (
    alt.Chart(heatmap_data)
    .mark_text(baseline='middle')
    .encode(
        x='category:N',
        y='emotion:N',
        text='count:Q',
        color=alt.condition(
            alt.datum.count > heatmap_data['count'].median(),
            alt.value('white'),
            alt.value('black')
        )
    )
)

(heatmap + text_labels).show()


alt.LayerChart(...)